# Chp 9.2 Case where the target block changes the number of output filters

**Case where the target block changes the number of output filters**

In [1]:
# from tensorflow import keras
# from tensorflow.keras import layers

# inputs = keras.Input(shape=(32, 32, 3))
# x = layers.Conv2D(32, 3, activation="relu")(inputs)
# residual = x
# x = layers.Conv2D(64, 3, activation="relu", padding="same")(x)
# residual = layers.Conv2D(64, 1)(residual)
# x = layers.add([x, residual])

# print(x)

**Case where the target block includes a max pooling layer**

In [5]:
# inputs = keras.Input(shape=(32, 32, 3))
# x = layers.Conv2D(32, 3, activation="relu")(inputs)
# residual = x
# x = layers.Conv2D(64, 3, activation="relu", padding="same")(x)
# x = layers.MaxPooling2D(2, padding="same")(x)
# residual = layers.Conv2D(64, 1, strides=2)(residual)
# x = layers.add([x, residual])

# print(x)

KerasTensor(type_spec=TensorSpec(shape=(None, 15, 15, 64), dtype=tf.float32, name=None), name='add_4/add:0', description="created by layer 'add_4'")


In [2]:
# inputs = keras.Input(shape=(32, 32, 3))
# x = layers.experimental.preprocessing.Rescaling(1./255)(inputs)

# def residual_block(x, filters, pooling=False):
#     residual = x
#     x = layers.Conv2D(filters, 3, activation="relu", padding="same")(x)
#     x = layers.Conv2D(filters, 3, activation="relu", padding="same")(x)
#     if pooling:
#         x = layers.MaxPooling2D(2, padding="same")(x)
#         residual = layers.Conv2D(filters, 1, strides=2)(residual)
#     elif filters != residual.shape[-1]:
#         residual = layers.Conv2D(filters, 1)(residual)
#     x = layers.add([x, residual])
#     return x

# x = residual_block(x, filters=32, pooling=True)
# x = residual_block(x, filters=64, pooling=True)
# x = residual_block(x, filters=128, pooling=False)

# x = layers.GlobalAveragePooling2D()(x)
# outputs = layers.Dense(1, activation="sigmoid")(x)
# model = keras.Model(inputs=inputs, outputs=outputs)
# model.summary()

### Batch normalization

### Depthwise separable convolutions

### Putting it together: a mini Xception-like model

In [ ]:
!rm -fr chp09_convnet

def init_datadir():
    from pathlib import Path
    import shutil
    import os

    def is_jupyter_lab():
        import re
        import psutil
        return any(re.search('jupyter-lab', x) for x in psutil.Process().parent().cmdline())
     
    if is_jupyter_lab():
        print("Using jupyter-lab. Click `Upload Files` in the file-explorer and upload `kaggle.json` manually")
        !kaggle competitions download -c dogs-vs-cats
        !unzip -qq dogs-vs-cats.zip
        !unzip -qq train.zip
    else:
        # colab
        from google.colab import files
        files.upload()
        !mkdir ~/.kaggle
        !cp kaggle.json ~/.kaggle/
        !chmod 600 ~/.kaggle/kaggle.json
        !ls /root/.kaggle
        !kaggle competitions download -c dogs-vs-cats --path ./chp09_convnet/src
        #!unzip -qq dogs-vs-cats.zip
        #!unzip -qq train.zip
        !unzip -qq ./chp09_convnet/src/train.zip -d ./chp09_convnet/data

    print("Finished downloading data..")


def listing9_3_5():
    import os, shutil, pathlib
    from tensorflow import keras
    from tensorflow.keras import layers
    from tensorflow.keras.preprocessing import image_dataset_from_directory

    # download data
    init_datadir()

    original_dir = pathlib.Path("chp09_convnet/data/train")
    new_base_dir = pathlib.Path("chp09_convnet/data/cats_vs_dogs_small")

    def make_subset(subset_name, start_index, end_index):
        for category in ("cat", "dog"):
            dir = new_base_dir / subset_name / category
            os.makedirs(dir, exist_ok = True)
            fnames = [f"{category}.{i}.jpg" for i in range(start_index, end_index)]
            for fname in fnames:
                shutil.copyfile(src=original_dir / fname,
                                dst=dir / fname)

    make_subset("train", start_index=0, end_index=1000)
    make_subset("validation", start_index=1000, end_index=1500)
    make_subset("test", start_index=1500, end_index=2500)

    train_dataset = image_dataset_from_directory(
        new_base_dir / "train",
        image_size=(180, 180),
        batch_size=32)
    validation_dataset = image_dataset_from_directory(
        new_base_dir / "validation",
        image_size=(180, 180),
        batch_size=32)
    test_dataset = image_dataset_from_directory(
        new_base_dir / "test",
        image_size=(180, 180),
        batch_size=32)

    ####################

    data_augmentation = keras.Sequential(
        [
            layers.experimental.preprocessing.RandomFlip("horizontal"),
            layers.experimental.preprocessing.RandomRotation(0.1),
            layers.experimental.preprocessing.RandomZoom(0.2),
        ]
    )

    ####################

    inputs = keras.Input(shape=(180, 180, 3))
    x = data_augmentation(inputs)


    x = layers.experimental.preprocessing.Rescaling(1./255)(x)
    x = layers.Conv2D(filters=32, kernel_size=5, use_bias=False)(x)

    for size in [32, 64, 128, 256, 512]:
        residual = x

        x = layers.BatchNormalization()(x)
        x = layers.Activation("relu")(x)
        x = layers.SeparableConv2D(size, 3, padding="same", use_bias=False)(x)

        x = layers.BatchNormalization()(x)
        x = layers.Activation("relu")(x)
        x = layers.SeparableConv2D(size, 3, padding="same", use_bias=False)(x)

        x = layers.MaxPooling2D(3, strides=2, padding="same")(x)

        residual = layers.Conv2D(
            size, 1, strides=2, padding="same", use_bias=False)(residual)
        x = layers.add([x, residual])

    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dropout(0.5)(x)
    outputs = layers.Dense(1, activation="sigmoid")(x)
    model = keras.Model(inputs=inputs, outputs=outputs)

    ####################

    model.compile(
        loss="binary_crossentropy",
        optimizer="rmsprop",
        metrics=["accuracy"]
    )


    # Fitting the model using a Dataset
    callbacks = [
        keras.callbacks.ModelCheckpoint(
            filepath="chp09_convnet/models/chp9_convnet_patterns.keras",
            save_best_only=True,
            monitor="val_loss"
        )
    ]

    history = model.fit(
        train_dataset,
        epochs=50,
        validation_data=validation_dataset,
        callbacks=callbacks,
    )

    print()
    print(history.history.keys())
    # dict_keys(['loss', 'accuracy', 'val_loss', 'val_accuracy'])

    ####################

listing9_3_5()